In [1]:
import pandas as pd
from rdkit import Chem
from rdkit.Chem import AllChem, Descriptors
from mordred import Calculator, descriptors
import numpy as np
from padelpy import from_smiles

In [2]:
df = pd.read_csv('Bioactivity_Cyclooxygenase-2_clean.csv')
df

,Unnamed: 0,Molecule ChEMBL ID,Molecular Weight,#RO5 Violations,AlogP,Smiles,Standard Value,pChEMBL Value,Ligand Efficiency BEI,Ligand Efficiency LE,Ligand Efficiency LLE,Ligand Efficiency SEI,Value
0,0,CHEMBL70104,364.46,0.0,3.49,CC1(C)OC(=O)C(OC2CCCCC2)=C1c1ccc(S(C)(=O)=O)cc1,40.0,7.40,20.30,0.40,3.91,10.62,0.04
1,1,CHEMBL96098,372.88,0.0,4.43,CCc1ccc(-c2ncc(Cl)cc2-c2ccc(S(C)(=O)=O)cc2)cn1,1700.0,5.77,15.47,0.32,1.34,9.63,1.70
2,2,CHEMBL97179,413.90,1.0,5.19,CCCCOC(=O)Cc1c(C)n(C(=O)c2ccc(Cl)cc2)c2ccc(OC)...,50.0,7.30,17.64,0.34,2.11,12.69,0.05
3,5,CHEMBL276436,316.49,1.0,5.66,CC(C)CCCC(=O)c1cc(C(C)(C)C)c2c(c1)C(C)(C)CO2,4500.0,5.35,16.89,0.32,-0.31,20.33,4.50
4,6,CHEMBL95380,574.08,2.0,5.28,COc1ccc2c(c1)c(CC(=O)NCCOC(=O)/C=C/c1ccc(N(C)C...,190.0,6.72,11.71,0.22,1.44,7.48,0.19
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3962,7903,CHEMBL52229,244.29,0.0,3.31,COc1ccc2ccc(=O)oc2c1CC=C(C)C,81100.0,4.09,16.75,0.31,0.78,10.37,81.10
3963,7939,CHEMBL189171,415.83,0.0,3.47,COc1ccc2c(c1)c(CC(=O)OCC(=O)O)c(C)n2C(=O)c1ccc...,10.0,8.00,19.24,0.38,4.53,8.44,10.00
3964,7950,CHEMBL4515706,394.51,1.0,6.00,COc1ccc(/C=C/c2c(CC=C(C)C)c(O)cc(O)c2CC=C(C)C)...,14700.0,4.83,12.25,0.23,-1.17,6.91,14.70
3965,7968,CHEMBL704,153.14,0.0,0.67,Nc1ccc(O)c(C(=O)O)c1,7530.0,5.12,33.45,0.64,4.45,6.13,7.53


In [9]:
# Расчёт дескрипторов из библиотеки RDKit и объединение с материнским датасетом
def smiles_desc_rdkit(df, smiles_column='Smiles'):

    result_df = df.copy()

    descriptors = {}

    iterator = df.iterrows()

    for idx, row in iterator:
        
        smiles = row[smiles_column]
        mol = Chem.MolFromSmiles(smiles)
        
        for desc_name, desc_funk in Descriptors.descList:
            n = desc_funk(mol)
            
            descriptors.setdefault(desc_name, []).append(n)
            
    descriptors_df = pd.DataFrame(descriptors, index=df.index)
    return pd.concat([df, descriptors_df], axis=1)  

In [10]:
df_rdkit_desc = smiles_desc_rdkit(df, smiles_column = 'Smiles')

In [11]:
df_rdkit_desc

,Unnamed: 0,Molecule ChEMBL ID,Molecular Weight,#RO5 Violations,AlogP,Smiles,Standard Value,pChEMBL Value,Ligand Efficiency BEI,Ligand Efficiency LE,...,fr_sulfide,fr_sulfonamd,fr_sulfone,fr_term_acetylene,fr_tetrazole,fr_thiazole,fr_thiocyan,fr_thiophene,fr_unbrch_alkane,fr_urea
0,0,CHEMBL70104,364.46,0.0,3.49,CC1(C)OC(=O)C(OC2CCCCC2)=C1c1ccc(S(C)(=O)=O)cc1,40.0,7.40,20.30,0.40,...,0,0,1,0,0,0,0,0,0,0
1,1,CHEMBL96098,372.88,0.0,4.43,CCc1ccc(-c2ncc(Cl)cc2-c2ccc(S(C)(=O)=O)cc2)cn1,1700.0,5.77,15.47,0.32,...,0,0,1,0,0,0,0,0,0,0
2,2,CHEMBL97179,413.90,1.0,5.19,CCCCOC(=O)Cc1c(C)n(C(=O)c2ccc(Cl)cc2)c2ccc(OC)...,50.0,7.30,17.64,0.34,...,0,0,0,0,0,0,0,0,1,0
3,5,CHEMBL276436,316.49,1.0,5.66,CC(C)CCCC(=O)c1cc(C(C)(C)C)c2c(c1)C(C)(C)CO2,4500.0,5.35,16.89,0.32,...,0,0,0,0,0,0,0,0,0,0
4,6,CHEMBL95380,574.08,2.0,5.28,COc1ccc2c(c1)c(CC(=O)NCCOC(=O)/C=C/c1ccc(N(C)C...,190.0,6.72,11.71,0.22,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3962,7903,CHEMBL52229,244.29,0.0,3.31,COc1ccc2ccc(=O)oc2c1CC=C(C)C,81100.0,4.09,16.75,0.31,...,0,0,0,0,0,0,0,0,0,0
3963,7939,CHEMBL189171,415.83,0.0,3.47,COc1ccc2c(c1)c(CC(=O)OCC(=O)O)c(C)n2C(=O)c1ccc...,10.0,8.00,19.24,0.38,...,0,0,0,0,0,0,0,0,0,0
3964,7950,CHEMBL4515706,394.51,1.0,6.00,COc1ccc(/C=C/c2c(CC=C(C)C)c(O)cc(O)c2CC=C(C)C)...,14700.0,4.83,12.25,0.23,...,0,0,0,0,0,0,0,0,0,0
3965,7968,CHEMBL704,153.14,0.0,0.67,Nc1ccc(O)c(C(=O)O)c1,7530.0,5.12,33.45,0.64,...,0,0,0,0,0,0,0,0,0,0


In [21]:
# Анализ дескрипторов имеющие пропуски значений для набора молекул
columns_with_nans = df_rdkit_desc.columns[df_rdkit_desc.isna().any()].tolist()
print("\nКолонки с пропусками:", columns_with_nans)


Колонки с пропусками: ['BCUT2D_MWHI', 'BCUT2D_MWLOW', 'BCUT2D_CHGHI', 'BCUT2D_CHGLO', 'BCUT2D_LOGPHI', 'BCUT2D_LOGPLOW', 'BCUT2D_MRHI', 'BCUT2D_MRLOW']


In [22]:
# Удаление дескрипторов с пропусками
df_rdkit_desc = df_rdkit_desc.drop(['BCUT2D_MWHI', 'BCUT2D_MWLOW', 'BCUT2D_CHGHI', 'BCUT2D_CHGLO', 'BCUT2D_LOGPHI', 'BCUT2D_LOGPLOW', 'BCUT2D_MRHI', 'BCUT2D_MRLOW'], axis=1)
columns_with_nans = df_rdkit_desc.columns[df_rdkit_desc.isna().any()].tolist()
print("\nКолонки с пропусками:", columns_with_nans)


Колонки с пропусками: []


In [23]:
df_rdkit_desc

,Unnamed: 0,Molecule ChEMBL ID,Molecular Weight,#RO5 Violations,AlogP,Smiles,Standard Value,pChEMBL Value,Ligand Efficiency BEI,Ligand Efficiency LE,...,fr_sulfide,fr_sulfonamd,fr_sulfone,fr_term_acetylene,fr_tetrazole,fr_thiazole,fr_thiocyan,fr_thiophene,fr_unbrch_alkane,fr_urea
0,0,CHEMBL70104,364.46,0.0,3.49,CC1(C)OC(=O)C(OC2CCCCC2)=C1c1ccc(S(C)(=O)=O)cc1,40.0,7.40,20.30,0.40,...,0,0,1,0,0,0,0,0,0,0
1,1,CHEMBL96098,372.88,0.0,4.43,CCc1ccc(-c2ncc(Cl)cc2-c2ccc(S(C)(=O)=O)cc2)cn1,1700.0,5.77,15.47,0.32,...,0,0,1,0,0,0,0,0,0,0
2,2,CHEMBL97179,413.90,1.0,5.19,CCCCOC(=O)Cc1c(C)n(C(=O)c2ccc(Cl)cc2)c2ccc(OC)...,50.0,7.30,17.64,0.34,...,0,0,0,0,0,0,0,0,1,0
3,5,CHEMBL276436,316.49,1.0,5.66,CC(C)CCCC(=O)c1cc(C(C)(C)C)c2c(c1)C(C)(C)CO2,4500.0,5.35,16.89,0.32,...,0,0,0,0,0,0,0,0,0,0
4,6,CHEMBL95380,574.08,2.0,5.28,COc1ccc2c(c1)c(CC(=O)NCCOC(=O)/C=C/c1ccc(N(C)C...,190.0,6.72,11.71,0.22,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3962,7903,CHEMBL52229,244.29,0.0,3.31,COc1ccc2ccc(=O)oc2c1CC=C(C)C,81100.0,4.09,16.75,0.31,...,0,0,0,0,0,0,0,0,0,0
3963,7939,CHEMBL189171,415.83,0.0,3.47,COc1ccc2c(c1)c(CC(=O)OCC(=O)O)c(C)n2C(=O)c1ccc...,10.0,8.00,19.24,0.38,...,0,0,0,0,0,0,0,0,0,0
3964,7950,CHEMBL4515706,394.51,1.0,6.00,COc1ccc(/C=C/c2c(CC=C(C)C)c(O)cc(O)c2CC=C(C)C)...,14700.0,4.83,12.25,0.23,...,0,0,0,0,0,0,0,0,0,0
3965,7968,CHEMBL704,153.14,0.0,0.67,Nc1ccc(O)c(C(=O)O)c1,7530.0,5.12,33.45,0.64,...,0,0,0,0,0,0,0,0,0,0


In [28]:
# Расчёт дескрипторов из библиотеки Mordred
def mordred_decs(df, smiles_column='Smiles', ignore_3D=True):
    calc = Calculator(descriptors, ignore_3D=ignore_3D)
    mord_desc = []

    for smi in df[smiles_column]:
        mol = Chem.MolFromSmiles(smi)
        desc = calc(mol)

        mord_desc.append(desc.asdict())

    mordred_df = pd.DataFrame(mord_desc)
    conc_desc = pd.concat([df, mordred_df], axis=1)
    return conc_desc

In [29]:
df_mordred_desc = mordred_decs(df)

In [38]:
df_mordred_desc

,Unnamed: 0,Molecule ChEMBL ID,Molecular Weight,#RO5 Violations,AlogP,Smiles,Standard Value,pChEMBL Value,Ligand Efficiency BEI,Ligand Efficiency LE,...,SRW10,TSRW10,MW,AMW,WPath,WPol,Zagreb1,Zagreb2,mZagreb1,mZagreb2
0,0,CHEMBL70104,364.46,0.0,3.49,CC1(C)OC(=O)C(OC2CCCCC2)=C1c1ccc(S(C)(=O)=O)cc1,40.0,7.40,20.30,0.40,...,10.277978,74.761225,364.134445,7.431315,1478,40,136.0,160.0,9.541667,5.208333
1,1,CHEMBL96098,372.88,0.0,4.43,CCc1ccc(-c2ncc(Cl)cc2-c2ccc(S(C)(=O)=O)cc2)cn1,1700.0,5.77,15.47,0.32,...,10.177894,59.944317,372.069926,8.858808,1536,40,132.0,154.0,8.840278,5.416667
2,2,CHEMBL97179,413.90,1.0,5.19,CCCCOC(=O)Cc1c(C)n(C(=O)c2ccc(Cl)cc2)c2ccc(OC)...,50.0,7.30,17.64,0.34,...,10.262315,79.000132,413.139386,7.795083,2264,47,148.0,175.0,10.361111,6.611111
3,5,CHEMBL276436,316.49,1.0,5.66,CC(C)CCCC(=O)c1cc(C(C)(C)C)c2c(c1)C(C)(C)CO2,4500.0,5.35,16.89,0.32,...,10.187500,71.969030,316.240230,5.749822,1179,36,122.0,142.0,10.541667,4.791667
4,6,CHEMBL95380,574.08,2.0,5.28,COc1ccc2c(c1)c(CC(=O)NCCOC(=O)/C=C/c1ccc(N(C)C...,190.0,6.72,11.71,0.22,...,10.546367,92.627598,573.203049,7.852097,7068,65,210.0,245.0,14.305556,9.138889
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3962,7903,CHEMBL52229,244.29,0.0,3.31,COc1ccc2ccc(=O)oc2c1CC=C(C)C,81100.0,4.09,16.75,0.31,...,9.758982,50.937658,244.109944,7.179704,588,27,90.0,104.0,6.666667,4.083333
3963,7939,CHEMBL189171,415.83,0.0,3.47,COc1ccc2c(c1)c(CC(=O)OCC(=O)O)c(C)n2C(=O)c1ccc...,10.0,8.00,19.24,0.38,...,10.280656,79.078849,415.082265,8.831538,2238,47,150.0,177.0,10.972222,6.444444
3964,7950,CHEMBL4515706,394.51,1.0,6.00,COc1ccc(/C=C/c2c(CC=C(C)C)c(O)cc(O)c2CC=C(C)C)...,14700.0,4.83,12.25,0.23,...,10.136621,64.059811,394.214409,6.681600,2318,44,142.0,162.0,11.861111,6.555556
3965,7968,CHEMBL704,153.14,0.0,0.67,Nc1ccc(O)c(C(=O)O)c1,7530.0,5.12,33.45,0.64,...,9.096387,41.025392,153.042593,8.502366,148,14,52.0,58.0,5.194444,2.472222


In [32]:
df_mordred_desc.to_csv('df_mordred_desc.csv')

In [39]:
# Анализ на пропуски
columns_with_nans = df_mordred_desc.columns[df_mordred_desc.isna().any()].tolist()
print("\nКолонки с пропусками:", columns_with_nans)


Колонки с пропусками: []


In [8]:
# Расчёт дескрипторов из библиотеки padelpy. Возникла ошибка Java JRE 6+ not found (required for PaDEL-Descriptor), хотя java jre установлен
smiles_series = df["Smiles"].astype(str)
smiles_list = smiles_series.tolist()

In [13]:
descriptors_df = from_smiles(smiles_list)

ReferenceError: Java JRE 6+ not found (required for PaDEL-Descriptor)

In [ ]:
df_padle_desc = pd.concat([df.reset_index(drop=True), descriptors_df], axis=1)

In [43]:
df_rdkit_desc = df_rdkit_desc.drop('Molecule ChEMBL ID', axis=1)

In [46]:
df_rdkit_desc = df_rdkit_desc.drop('Smiles', axis=1)

In [47]:
# Расчёт корреляций между признакми для удаления сильно коррелирующих фичей
def del_corr(df, threshold=0.7):
    corr_matr = df.corr().abs()
    upper = corr_matr.where(np.triu(np.ones(corr_matr.shape), k=1).astype(bool))
    to_drop = [column for column in upper.columns if any(upper[column] > threshold)]

    return df.drop(columns=to_drop)

In [48]:
df_rdkit_desc = del_corr(df_rdkit_desc, threshold=0.7)
df_rdkit_desc

,Unnamed: 0,Molecular Weight,#RO5 Violations,AlogP,Standard Value,pChEMBL Value,Ligand Efficiency SEI,Value,MaxAbsEStateIndex,MinAbsEStateIndex,...,fr_quatN,fr_sulfide,fr_sulfone,fr_term_acetylene,fr_tetrazole,fr_thiazole,fr_thiocyan,fr_thiophene,fr_unbrch_alkane,fr_urea
0,0,364.46,0.0,3.49,40.0,7.40,10.62,0.04,12.372552,0.037493,...,0,0,1,0,0,0,0,0,0,0
1,1,372.88,0.0,4.43,1700.0,5.77,9.63,1.70,11.641862,0.279401,...,0,0,1,0,0,0,0,0,0,0
2,2,413.90,1.0,5.19,50.0,7.30,12.69,0.05,13.229344,0.097334,...,0,0,0,0,0,0,0,0,1,0
3,5,316.49,1.0,5.66,4500.0,5.35,20.33,4.50,12.675413,0.024250,...,0,0,0,0,0,0,0,0,0,0
4,6,574.08,2.0,5.28,190.0,6.72,7.48,0.19,13.438530,0.025891,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3962,7903,244.29,0.0,3.31,81100.0,4.09,10.37,81.10,11.350518,0.338701,...,0,0,0,0,0,0,0,0,0,0
3963,7939,415.83,0.0,3.47,10.0,8.00,8.44,10.00,13.155765,0.176906,...,0,0,0,0,0,0,0,0,0,0
3964,7950,394.51,1.0,6.00,14700.0,4.83,6.91,14.70,10.501451,0.058273,...,0,0,0,0,0,0,0,0,0,0
3965,7968,153.14,0.0,0.67,7530.0,5.12,6.13,7.53,10.358426,0.175926,...,0,0,0,0,0,0,0,0,0,0


In [49]:
df_mordred_desc = df_mordred_desc.drop(['Molecule ChEMBL ID', 'Smiles'], axis=1)

In [50]:
df_mordred_desc = del_corr(df_mordred_desc, threshold=0.7)
df_mordred_desc

,Unnamed: 0,Molecular Weight,#RO5 Violations,AlogP,Standard Value,pChEMBL Value,Ligand Efficiency SEI,Value,ABC,ABCGG,...,n6FARing,n10FARing,n4FAHRing,n5FAHRing,n6FAHRing,JGI6,JGI7,JGI8,JGI9,JGI10
0,0,364.46,0.0,3.49,40.0,7.40,10.62,0.04,module 'numpy' has no attribute 'float'.\n`np....,module 'numpy' has no attribute 'float'.\n`np....,...,0,0,0,0,0,0.022068,0.019905,0.011665,0.011875,0.010101
1,1,372.88,0.0,4.43,1700.0,5.77,9.63,1.70,module 'numpy' has no attribute 'float'.\n`np....,module 'numpy' has no attribute 'float'.\n`np....,...,0,0,0,0,0,0.020816,0.017297,0.019684,0.009643,0.007513
2,2,413.90,1.0,5.19,50.0,7.30,12.69,0.05,module 'numpy' has no attribute 'float'.\n`np....,module 'numpy' has no attribute 'float'.\n`np....,...,0,0,0,0,0,0.018910,0.015313,0.011976,0.006887,0.008903
3,5,316.49,1.0,5.66,4500.0,5.35,20.33,4.50,module 'numpy' has no attribute 'float'.\n`np....,module 'numpy' has no attribute 'float'.\n`np....,...,0,0,0,0,0,0.014749,0.015029,0.014272,0.022043,0.002857
4,6,574.08,2.0,5.28,190.0,6.72,7.48,0.19,module 'numpy' has no attribute 'float'.\n`np....,module 'numpy' has no attribute 'float'.\n`np....,...,0,0,0,0,0,0.018553,0.013535,0.010708,0.007536,0.007397
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3962,7903,244.29,0.0,3.31,81100.0,4.09,10.37,81.10,module 'numpy' has no attribute 'float'.\n`np....,module 'numpy' has no attribute 'float'.\n`np....,...,0,0,0,0,0,0.023545,0.017650,0.010204,0.000000,0.000000
3963,7939,415.83,0.0,3.47,10.0,8.00,8.44,10.00,module 'numpy' has no attribute 'float'.\n`np....,module 'numpy' has no attribute 'float'.\n`np....,...,0,0,0,0,0,0.019349,0.016493,0.012178,0.008462,0.010315
3964,7950,394.51,1.0,6.00,14700.0,4.83,6.91,14.70,module 'numpy' has no attribute 'float'.\n`np....,module 'numpy' has no attribute 'float'.\n`np....,...,0,0,0,0,0,0.017711,0.015708,0.012869,0.011504,0.008424
3965,7968,153.14,0.0,0.67,7530.0,5.12,6.13,7.53,module 'numpy' has no attribute 'float'.\n`np....,module 'numpy' has no attribute 'float'.\n`np....,...,0,0,0,0,0,0.000000,0.000000,0.000000,0.000000,0.000000


In [54]:
# Расчёт дисперсии для дескрипторов, и удаление тех у кого она нулевая
def zero_var(df):
    var = df.var(numeric_only=True)
    
    col = var[var == 0].index.tolist()
    
    return df.drop(columns=col)

In [55]:
df_rdkit_desc = zero_var(df_rdkit_desc)
df_rdkit_desc

,Unnamed: 0,Molecular Weight,#RO5 Violations,AlogP,Standard Value,pChEMBL Value,Ligand Efficiency SEI,Value,MaxAbsEStateIndex,MinAbsEStateIndex,...,fr_pyridine,fr_sulfide,fr_sulfone,fr_term_acetylene,fr_tetrazole,fr_thiazole,fr_thiocyan,fr_thiophene,fr_unbrch_alkane,fr_urea
0,0,364.46,0.0,3.49,40.0,7.40,10.62,0.04,12.372552,0.037493,...,0,0,1,0,0,0,0,0,0,0
1,1,372.88,0.0,4.43,1700.0,5.77,9.63,1.70,11.641862,0.279401,...,2,0,1,0,0,0,0,0,0,0
2,2,413.90,1.0,5.19,50.0,7.30,12.69,0.05,13.229344,0.097334,...,0,0,0,0,0,0,0,0,1,0
3,5,316.49,1.0,5.66,4500.0,5.35,20.33,4.50,12.675413,0.024250,...,0,0,0,0,0,0,0,0,0,0
4,6,574.08,2.0,5.28,190.0,6.72,7.48,0.19,13.438530,0.025891,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3962,7903,244.29,0.0,3.31,81100.0,4.09,10.37,81.10,11.350518,0.338701,...,0,0,0,0,0,0,0,0,0,0
3963,7939,415.83,0.0,3.47,10.0,8.00,8.44,10.00,13.155765,0.176906,...,0,0,0,0,0,0,0,0,0,0
3964,7950,394.51,1.0,6.00,14700.0,4.83,6.91,14.70,10.501451,0.058273,...,0,0,0,0,0,0,0,0,0,0
3965,7968,153.14,0.0,0.67,7530.0,5.12,6.13,7.53,10.358426,0.175926,...,0,0,0,0,0,0,0,0,0,0


In [56]:
df_mordred_desc = zero_var(df_mordred_desc)
df_mordred_desc

,Unnamed: 0,Molecular Weight,#RO5 Violations,AlogP,Standard Value,pChEMBL Value,Ligand Efficiency SEI,Value,ABC,ABCGG,...,n12FRing,n8FaRing,n12FaRing,nG12FaRing,n10FARing,JGI6,JGI7,JGI8,JGI9,JGI10
0,0,364.46,0.0,3.49,40.0,7.40,10.62,0.04,module 'numpy' has no attribute 'float'.\n`np....,module 'numpy' has no attribute 'float'.\n`np....,...,0,0,0,0,0,0.022068,0.019905,0.011665,0.011875,0.010101
1,1,372.88,0.0,4.43,1700.0,5.77,9.63,1.70,module 'numpy' has no attribute 'float'.\n`np....,module 'numpy' has no attribute 'float'.\n`np....,...,0,0,0,0,0,0.020816,0.017297,0.019684,0.009643,0.007513
2,2,413.90,1.0,5.19,50.0,7.30,12.69,0.05,module 'numpy' has no attribute 'float'.\n`np....,module 'numpy' has no attribute 'float'.\n`np....,...,0,0,0,0,0,0.018910,0.015313,0.011976,0.006887,0.008903
3,5,316.49,1.0,5.66,4500.0,5.35,20.33,4.50,module 'numpy' has no attribute 'float'.\n`np....,module 'numpy' has no attribute 'float'.\n`np....,...,0,0,0,0,0,0.014749,0.015029,0.014272,0.022043,0.002857
4,6,574.08,2.0,5.28,190.0,6.72,7.48,0.19,module 'numpy' has no attribute 'float'.\n`np....,module 'numpy' has no attribute 'float'.\n`np....,...,0,0,0,0,0,0.018553,0.013535,0.010708,0.007536,0.007397
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3962,7903,244.29,0.0,3.31,81100.0,4.09,10.37,81.10,module 'numpy' has no attribute 'float'.\n`np....,module 'numpy' has no attribute 'float'.\n`np....,...,0,0,0,0,0,0.023545,0.017650,0.010204,0.000000,0.000000
3963,7939,415.83,0.0,3.47,10.0,8.00,8.44,10.00,module 'numpy' has no attribute 'float'.\n`np....,module 'numpy' has no attribute 'float'.\n`np....,...,0,0,0,0,0,0.019349,0.016493,0.012178,0.008462,0.010315
3964,7950,394.51,1.0,6.00,14700.0,4.83,6.91,14.70,module 'numpy' has no attribute 'float'.\n`np....,module 'numpy' has no attribute 'float'.\n`np....,...,0,0,0,0,0,0.017711,0.015708,0.012869,0.011504,0.008424
3965,7968,153.14,0.0,0.67,7530.0,5.12,6.13,7.53,module 'numpy' has no attribute 'float'.\n`np....,module 'numpy' has no attribute 'float'.\n`np....,...,0,0,0,0,0,0.000000,0.000000,0.000000,0.000000,0.000000
